In [1]:
# Librería para calcular de forma fácil la exactitud

%pip install torchmetrics --quiet

Note: you may need to restart the kernel to use updated packages.


Usando una librería de python (Keras o Pytorch) desarrolle los modelos de Convolutional Neural Networks que se describen para cada
problema en particular (usar una jupyter notebook por cada dataset y entregar todos en una carpeta zip).


1. MNIST Dataset. Escalar las imágenes en un rango de [0,1]. Usar 10% del conjunto de prueba para validación en el entrenamiento. Usar
como función de perdida categorical_crossentropy, optimizador adam y métrica exactitud durante el entrenamiento de 15 épocas. Aplicar
zero-padding en las capas conv2d. Reportar valor de perdida y exactitud para entrenamiento y prueba. Tomando como referencia el
siguiente diagrama:

1. Escalar las imágenes en un rango de [0,1]. 


In [2]:
import torchvision.datasets as datasets
from torchvision import transforms

# Normalizamos las imágenes en rango de 0 y 1
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Cargar dataset y normalizar
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [3]:
next(iter(mnist_trainset))

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

2. Usar 10% del conjunto de prueba para validación en el entrenamiento. 


In [4]:
from torch.utils.data import random_split

# Usar 10% del conjunto de prueba para validación en el entrenamiento. 
test_dataset_size = len(mnist_testset)

validation_size = int(test_dataset_size * 0.1)
test_size = test_dataset_size - validation_size

mnist_validationset, mnist_testset = random_split(mnist_testset, [validation_size, test_size])

print(f"Tamaño del conjunto de entrenamiento: {len(mnist_trainset)}")
print(f"Tamaño del conjunto de prueba: {len(mnist_testset)}")
print(f"Tamaño del conjunto de validación: {len(mnist_validationset)}")

Tamaño del conjunto de entrenamiento: 60000
Tamaño del conjunto de prueba: 9000
Tamaño del conjunto de validación: 1000


In [5]:
from torch.utils.data import DataLoader

batch_size = 64

# Crear los conjuntos de entrenamiento, prueba y validación
train_data = DataLoader(dataset=mnist_trainset, 
                        batch_size=batch_size,
                        num_workers=2,
                        shuffle=True)

test_data = DataLoader(dataset=mnist_trainset, 
                        batch_size=batch_size,
                        num_workers=2)

val_data = DataLoader(dataset=mnist_validationset, 
                        batch_size=batch_size,
                        num_workers=2)

![Red](./variety/red.png)

4. Aplicar zero-padding en las capas conv2d. Reportar valor de perdida y exactitud para entrenamiento y prueba.

a) Implemente la CNN del diagrama omitiendo las capas dropout e implemente EarlyStopping monitoreando la perdida del conjunto de
validación y una paciencia de 2.



In [48]:
import torch
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self, num_classes):
        super(NeuralNetwork, self).__init__()

        self.network = nn.Sequential(
            nn.ZeroPad2d(1),  # Add zero-padding of size 1
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            nn.ZeroPad2d(1),  # Add zero-padding of size 1
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Flatten(),
            nn.Linear(in_features=12544, out_features=128),
            nn.Linear(in_features=128, out_features=num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.network(x)


model = NeuralNetwork(num_classes=10)
print(model)

NeuralNetwork(
  (network): Sequential(
    (0): ZeroPad2d((1, 1, 1, 1))
    (1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (2): ReLU()
    (3): ZeroPad2d((1, 1, 1, 1))
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Flatten(start_dim=1, end_dim=-1)
    (8): Linear(in_features=12544, out_features=128, bias=True)
    (9): Linear(in_features=128, out_features=10, bias=True)
    (10): Softmax(dim=1)
  )
)


In [49]:
import torch

def get_device():
    device = ""
    if torch.cuda.is_available():
        !nvidia-smi
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    return device

device = get_device()
device

Mon May  8 01:19:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        Off  | 00000000:00:05.0 Off |                  Off |
| 26%   27C    P8     8W / 180W |    931MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

device(type='cuda')

In [50]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [53]:
import numpy as np

from pytorchtools import EarlyStopping
from torchmetrics.classification import Accuracy

def train(model, patience, epochs, num_classes):
    train_losses = []
    valid_losses = []

    avg_train_losses = []
    avg_valid_losses = []

    early_stopping = EarlyStopping(patience=patience, verbose=True)

    accuracy = Accuracy(task="multiclass", num_classes=num_classes)

    model = model.to(device)

    for epoch in range(epochs):
        running_loss = 0.0
        running_val_loss = 0.0
        model.train()
        for batch, (images, labels) in enumerate(train_data):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            train_losses.append(loss.item())

            if batch % 500 == 0:
                print(f'Epoch: {epoch+1}/{epochs} | Batch: {batch+1}/{len(train_data)} | Loss: {running_loss/100:.4f}')
                running_loss = 0.0

        # Validación del model
        model.eval()
        with torch.no_grad():
            for _, (images, labels) in enumerate(val_data):
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)

                _, predicted = torch.max(outputs, 1)

                loss = criterion(outputs, labels)

                accuracy.update(predicted.to('cpu'), labels.to('cpu'))

                running_val_loss += loss.item()
                valid_losses.append(loss.item())

        acc = accuracy.compute()
        print(f"Running Accuracy: {acc}")

        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)

        print(f'Epoch: {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f}')

        train_losses = []
        valid_losses = []

        early_stopping(valid_loss, model)

        if early_stopping.early_stop:
            print(f'Early stopping at epoch {epoch+1}')
            break


    model.load_state_dict(torch.load('checkpoint.pt'))

    print(f"Accuracy promedio: {accuracy.compute()}")

    return model, avg_train_losses, avg_valid_losses

In [54]:
patience = 2
epochs = 15
num_classes = 10

model, train_loss, valid_loss = train(model, 
                                      patience, 
                                      epochs, 
                                      num_classes)

Epoch: 1/15 | Batch: 1/938 | Loss: 0.0148
Epoch: 1/15 | Batch: 501/938 | Loss: 7.3892
Running Accuracy: 0.9789999723434448
Epoch: 1/15 | Train Loss: 1.4789 | Valid Loss: 1.4827
Validation loss decreased (inf --> 1.482673).  Saving model ...
Epoch: 2/15 | Batch: 1/938 | Loss: 0.0149
Epoch: 2/15 | Batch: 501/938 | Loss: 7.3815
Running Accuracy: 0.9789999723434448
Epoch: 2/15 | Train Loss: 1.4769 | Valid Loss: 1.4813
Validation loss decreased (1.482673 --> 1.481316).  Saving model ...
Epoch: 3/15 | Batch: 1/938 | Loss: 0.0148
Epoch: 3/15 | Batch: 501/938 | Loss: 7.3745
Running Accuracy: 0.9803333282470703
Epoch: 3/15 | Train Loss: 1.4756 | Valid Loss: 1.4777
Validation loss decreased (1.481316 --> 1.477736).  Saving model ...
Epoch: 4/15 | Batch: 1/938 | Loss: 0.0149
